In [ ]:

TASKID = taskID
INPUT_SIZE = (256, 256)

df = spark.sql("SELECT * FROM InFreGen.pic WHERE taskID = {}".format(taskID))

StatementMeta(, , , Waiting, , Waiting)

DataFrame[taskID: int, picID: int, Resolution: string, sizeChoice: string, operations: string, operationsReturn: string, url: string, originalPicPath: string, State: int, curPicPath: string, finalPicPath: string, keywords: string]

In [ ]:
from PIL import Image
from delta.tables import DeltaTable
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# update the pic table after resizing
def update_afterResize(taskID, picID, target_size, curPicPath):
    delta_table = DeltaTable.forName(spark, "InFreGen.pic")
    # resize the image
    try:
        with Image.open(curPicPath) as img:
            img_resized = img.resize(target_size)
            resized_pic_path = curPicPath.replace(".jpg", "_resized.jpg")  # save the resized image
            img_resized.save(resized_pic_path)
    except Exception as e:
        print(f"Error resizing image {curPicPath}: {e}")
        return

    resolution_str = f"{target_size[0]}*{target_size[1]}"
    
    spark.sql(f"UPDATE InFreGen.pic SET curPicPath = '{resized_pic_path}', state = '5' WHERE taskID = '{taskID}' AND picID = '{picID}' ")
    print(f"Updated table for taskID: {taskID}, picID: {picID}")


StatementMeta(, , , Waiting, , Waiting)

In [ ]:
import json

data = df.select("taskID", "picID", "curPicPath","State").toPandas()

for index, row in data.iterrows():

    taskID = row["taskID"]
    picID = row["picID"]
    curPicPath = row["curPicPath"]

    update_afterResize(taskID, picID, INPUT_SIZE, curPicPath)


StatementMeta(, , , Waiting, , Waiting)

Updated table for taskID: 23, picID: 0


Updated table for taskID: 23, picID: 4


Updated table for taskID: 23, picID: 3


Updated table for taskID: 23, picID: 2


Updated table for taskID: 23, picID: 1
